# Em's VCF pipeline  
This is a workbook to take reads, align to a reference and generate a vcf.  
I hope the experince here will allow some cells to be reused in other pipelines with downstream software.  

## 0. Software installations and compute setup    
I am going to try and put everyting in the **conda env:Bondlab_phylo_env**   
If that doesn't work I will have to make a new env - but thats the plan for today.   
The below cell checks the number of cpus available - if thats not enough - quit this session and ask for more.

In [1]:
# check cpus available with "cheeky oneliner"
import os; print("CPUs available:", int(os.getenv("SLURM_CPUS_PER_TASK") or (lambda q,p: os.cpu_count() if q=="max" else max(1, int(int(q)/int(p))))(*open("/sys/fs/cgroup/cpu.max").read().split())))

CPUs available: 64


**Is that enough cpus? - If not restart onDemand with more!**

# 1. Input data and output directory  
The minium input data is a reference and a read set, but the difficulty here is that spider genome sizes are usually human sized or a bit bigger, and that you don't just have 1 read sequence file - but a directory of them with forward and rev reads for each sample/individual.   

_**Naming**_   
_Please_ name the input read files something useful and for now I will try and keep the stem name of the input files intact throughout the process. Perhaps add the genus_species_sample_identifiyer to the sequence file name that came from the sequence provider, and live with the crazy long names. 

**INSTRUCTIONS!!!**   
1. **If your reference fasta file is not in    /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/data/references  PLEASE COPY IT THERE!**
2.  **If your reads are not in a directory in **_Ems_vcf_pipeline/data/read_directories PLEASE MAKE A NEW DIRECTORY FOR YOUR SEQUENCE READS AND SOFTLINK THEM THERE (i.e. ln -s read_name new_directory) TO SAVE STORAGE**
3. Then use the below cell to select both the reference and the new read directory and start the analysis in the cells below.  

In [2]:
import os
import re
from ipywidgets import Dropdown, Button, VBox, HTML
from IPython.display import display, clear_output

# --- Paths (adjust if you relocate project) ---
PROJECT_ROOT = "/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline"
refs_dir     = f"{PROJECT_ROOT}/data/references"
read_dirs_dir= f"{PROJECT_ROOT}/data/read_directories"

# --- List available references and read directories ---
def list_references(path):
    # Allow .fa, .fasta, .fna; ignore .fai/.dict etc.
    valid_ext = (".fa", ".fasta", ".fna")
    refs = [f for f in os.listdir(path) 
            if f.lower().endswith(valid_ext) and os.path.isfile(os.path.join(path, f))]
    return sorted(refs)

def list_read_dirs(path):
    return sorted([d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))])

references = list_references(refs_dir)
read_dirs  = list_read_dirs(read_dirs_dir)

# --- Widgets ---
ref_dropdown      = Dropdown(options=references, description="Reference:", layout={'width':'70%'})
read_dir_dropdown = Dropdown(options=read_dirs,  description="Read set:",  layout={'width':'70%'})
confirm_button    = Button(description="Confirm selection", button_style="success")
msg               = HTML()

display(VBox([ref_dropdown, read_dir_dropdown, confirm_button, msg]))

# --- Globals populated on confirm ---
selected_ref = None
selected_read_dir = None
selected_reads = []       # all FASTQ files
read_format = None        # 'fastq.gz' or 'fastq'
paired_samples = {}       # sample_id -> {'R1': path, 'R2': path}
unpaired = []             # list of files that could not be paired
problems = []             # any issues to report

# --- Helpers ---

# Recognize R1/R2 patterns and derive a sample ID that groups pairs
R1_PATTERNS = [r'(_R?)1(_|\.|$)', r'(\.R?)1(_|\.|$)']
R2_PATTERNS = [r'(_R?)2(_|\.|$)', r'(\.R?)2(_|\.|$)']

def is_fastq(fname):
    low = fname.lower()
    return low.endswith(".fastq") or low.endswith(".fq") or low.endswith(".fastq.gz") or low.endswith(".fq.gz")

def fastq_suffix(fname):
    low = fname.lower()
    if low.endswith(".fastq.gz"): return "fastq.gz"
    if low.endswith(".fq.gz"):     return "fq.gz"
    if low.endswith(".fastq"):     return "fastq"
    if low.endswith(".fq"):        return "fq"
    return None

def detect_read_role_and_stem(basename):
    """
    Return (role, sample_stem) where role in {'R1','R2',None}.
    sample_stem is the basename with the R1/R2 token removed to group pairs.
    """
    name = basename
    name_no_ext = re.sub(r'(\.fastq|\.fq)(\.gz)?$', '', name, flags=re.IGNORECASE)

    # Try R1 first
    for pat in R1_PATTERNS:
        m = re.search(pat, name_no_ext, flags=re.IGNORECASE)
        if m:
            stem = name_no_ext[:m.start()] + name_no_ext[m.end():]
            return "R1", stem

    # Then R2
    for pat in R2_PATTERNS:
        m = re.search(pat, name_no_ext, flags=re.IGNORECASE)
        if m:
            stem = name_no_ext[:m.start()] + name_no_ext[m.end():]
            return "R2", stem

    return None, name_no_ext  # role unknown

def pair_reads(fastq_files):
    """
    Given a list of fastq file paths, return:
      paired: dict sample_id -> {'R1': path, 'R2': path}
      unpaired: list of paths that couldn't be paired
    Groups by derived sample_stem (see detect_read_role_and_stem).
    """
    groups = {}
    for f in fastq_files:
        base = os.path.basename(f)
        role, stem = detect_read_role_and_stem(base)
        groups.setdefault(stem, {}).setdefault('files', []).append((role, f))

    paired = {}
    unpaired_files = []
    for stem, info in groups.items():
        r1 = [p for role, p in info['files'] if role == 'R1']
        r2 = [p for role, p in info['files'] if role == 'R2']

        if len(r1) == 1 and len(r2) == 1:
            paired[stem] = {'R1': r1[0], 'R2': r2[0]}
        else:
            # Any files for this stem that aren't a clean pair are considered unpaired
            unpaired_files.extend([p for _, p in info['files']])

    return paired, sorted(unpaired_files)

def summary_html(ref_path, read_dir, fq_files, fmt, paired, unpaired, issues):
    n_fastq = len(fq_files)
    n_samples = len(paired)
    fmt_str = fmt if fmt else "Unknown"
    problems = list(issues)

    if n_fastq == 0:
        problems.append("No FASTQ files detected in the selected read directory.")
    if fmt is None:
        problems.append("Read format could not be determined.")
    if unpaired:
        problems.append(f"{len(unpaired)} file(s) could not be paired as R1/R2.")
    if n_samples == 0 and n_fastq > 0:
        problems.append("No valid R1/R2 pairs detected.")

    color = "#2e7d32" if not problems else "#b71c1c"

    issues_html = ""
    if problems:
        issues_html = "<ul style='margin:4px 0; padding-left:18px;'>" + "".join([f"<li>{p}</li>" for p in problems[:10]]) + "</ul>"

    return f"""
    <div style="border:1px solid {color}; padding:6px; border-radius:6px; font-size:14px; line-height:1.4;">
      <div style="margin:0;">
        <b>Reference file:</b> <code>{ref_path}</code><br>
        <b>Read directory:</b> <code>{read_dir}</code><br>
        <b>Read format:</b> <code>{fmt_str}</code><br>
        <b>FASTQ files:</b> {n_fastq} &nbsp;|&nbsp; <b>Samples:</b> {n_samples}
      </div>
      {"<hr style='margin:6px 0;'><b>Issues:</b>" + issues_html if problems else "<p style='color:#2e7d32; margin:4px 0;'><b>All good.</b> Reads look properly paired.</p>"}
    </div>
    """
    
def on_confirm(_):
    clear_output(wait=True)
    display(VBox([ref_dropdown, read_dir_dropdown, confirm_button, msg]))

    # Reset globals
    global selected_ref, selected_read_dir, selected_reads, read_format, paired_samples, unpaired, problems
    selected_ref = os.path.join(refs_dir, ref_dropdown.value) if ref_dropdown.value else None
    selected_read_dir = os.path.join(read_dirs_dir, read_dir_dropdown.value) if read_dir_dropdown.value else None
    selected_reads = []
    read_format = None
    paired_samples = {}
    unpaired = []
    problems = []

    # Validate presence
    if not selected_ref or not os.path.exists(selected_ref):
        problems.append("Selected reference file is missing.")
    if not selected_read_dir or not os.path.isdir(selected_read_dir):
        problems.append("Selected read directory is missing.")

    # Gather FASTQ files in the selected directory
    fastqs = []
    other_files = []
    if selected_read_dir and os.path.isdir(selected_read_dir):
        for f in sorted(os.listdir(selected_read_dir)):
            path = os.path.join(selected_read_dir, f)
            if os.path.isfile(path):
                if is_fastq(f):
                    fastqs.append(path)
                else:
                    other_files.append(path)
    selected_reads = fastqs

    # Determine read format consistency
    exts = {fastq_suffix(os.path.basename(p)) for p in fastqs}
    exts.discard(None)
    if len(exts) == 0:
        read_format = None
    elif len(exts) == 1:
        only = exts.pop()
        # Normalize fq vs fastq
        read_format = "fastq.gz" if "gz" in only else "fastq"
    else:
        problems.append(f"Mixed FASTQ extensions detected: {', '.join(sorted(exts))}. Please standardize.")
        # Best guess for display
        read_format = "/".join(sorted(exts))

    # Pair reads
    paired_samples, unpaired = pair_reads(fastqs)
    # Build and display summary
    msg.value = summary_html(selected_ref, selected_read_dir, fastqs, read_format, paired_samples, unpaired, problems)
    
confirm_button.on_click(on_confirm)

In [3]:
import os, re, shutil, math
from IPython.display import display, HTML

# -------------------------
# Inputs from earlier cells
# -------------------------
try:
    PROJECT_ROOT
except NameError:
    PROJECT_ROOT = "/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline"

# These should be set by your selection cell:
#   REFERENCE: path to FASTA
#   READ_DIR:  selected read directory
#   READS:     list of fastq paths
#   PAIRS:     dict of paired samples
#   FORMAT:    'fastq' or 'fastq.gz'
#   results_dir: desired output dir
# If not set yet, we still run with Nones.
globals_dict = globals()
REFERENCE   = globals_dict.get("selected_ref", globals_dict.get("REFERENCE"))
READ_DIR    = globals_dict.get("selected_read_dir", globals_dict.get("READ_DIR"))
READS       = globals_dict.get("selected_reads", globals_dict.get("READS", []))
PAIRS       = globals_dict.get("paired_samples", globals_dict.get("PAIRS", {}))
FORMAT      = globals_dict.get("read_format", globals_dict.get("FORMAT"))
results_dir = globals_dict.get("results_dir", f"{PROJECT_ROOT}/results/UNKNOWN_READSET")

# -------------------------
# Helpers to read cgroup/Slurm/system limits
# -------------------------

def _read_first_existing(path_list):
    for p in path_list:
        try:
            with open(p, "r") as fh:
                return fh.read().strip()
        except Exception:
            pass
    return None

def detect_cpus():
    # Start with Python's view
    py_cpu = os.cpu_count() or 1

    # Slurm envs (common)
    slurm_cpus_task = os.getenv("SLURM_CPUS_PER_TASK")
    slurm_cpus_node = os.getenv("SLURM_CPUS_ON_NODE")

    candidates = [py_cpu]
    for v in (slurm_cpus_task, slurm_cpus_node):
        try:
            if v is not None:
                candidates.append(int(v))
        except ValueError:
            pass

    # cgroup v2 cpu.max (format: "quota period" in microseconds; "max" if unlimited)
    cpu_max = _read_first_existing([
        "/sys/fs/cgroup/cpu.max",                       # cgroup v2
        "/sys/fs/cgroup/cpu/cpu.cfs_quota_us"           # cgroup v1
    ])

    # cgroup v2: cpu.max "max <period>" means no limit; otherwise cpus = quota/period
    # cgroup v1: cpu.cfs_quota_us + cpu.cfs_period_us
    if cpu_max and " " in cpu_max:
        quota_str, period_str = cpu_max.split()
        if quota_str != "max":
            try:
                quota = int(quota_str); period = int(period_str)
                cg_cpus = max(1, int(quota / period))
                candidates.append(cg_cpus)
            except Exception:
                pass
    else:
        # cgroup v1 pair
        quota_v1 = _read_first_existing([
            "/sys/fs/cgroup/cpu/cpu.cfs_quota_us",
        ])
        period_v1 = _read_first_existing([
            "/sys/fs/cgroup/cpu/cpu.cfs_period_us",
        ])
        try:
            if quota_v1 and period_v1:
                quota = int(quota_v1); period = int(period_v1)
                if quota > 0:
                    candidates.append(max(1, int(quota/period)))
        except Exception:
            pass

    return max(1, min(candidates))

def detect_mem_bytes():
    # Prefer cgroup v2 memory.max (or v1 memory.limit_in_bytes)
    val = _read_first_existing([
        "/sys/fs/cgroup/memory.max",                            # cgroup v2
        "/sys/fs/cgroup/memory/memory.limit_in_bytes"          # cgroup v1
    ])
    if val:
        try:
            # cgroup v2 can return "max"
            if val.strip() != "max":
                m = int(val)
                # Often memory.high also exists; we stick with hard limit
                return m
        except Exception:
            pass

    # Fall back to MemTotal from /proc/meminfo
    try:
        with open("/proc/meminfo", "r") as fh:
            for line in fh:
                if line.startswith("MemTotal:"):
                    parts = line.split()
                    # MemTotal: <kB>
                    kb = int(parts[1])
                    return kb * 1024
    except Exception:
        pass
    return None

def human_bytes(n):
    if n is None:
        return "Unknown"
    units = ["B","KB","MB","GB","TB","PB"]
    i = 0
    f = float(n)
    while f >= 1024 and i < len(units)-1:
        f/=1024; i+=1
    if i <= 2:
        return f"{f:.0f} {units[i]}"
    return f"{f:.2f} {units[i]}"

def dir_size_bytes(path):
    total = 0
    for root, _, files in os.walk(path):
        for f in files:
            try:
                total += os.path.getsize(os.path.join(root, f))
            except Exception:
                pass
    return total

def list_file_sizes(paths):
    total = 0
    details = []
    for p in paths:
        try:
            sz = os.path.getsize(p)
            total += sz
            details.append((p, sz))
        except Exception:
            details.append((p, None))
    return total, details

# -------------------------
# Collect environment info
# -------------------------

cpu_count  = detect_cpus()
mem_bytes  = detect_mem_bytes()
mem_str    = human_bytes(mem_bytes) if mem_bytes else "Unknown"

# Slurm environment (if any)
slurm_jobid  = os.getenv("SLURM_JOB_ID")
slurm_nodel  = os.getenv("SLURM_NODELIST")
slurm_cpt    = os.getenv("SLURM_CPUS_PER_TASK")
slurm_con    = os.getenv("SLURM_CPUS_ON_NODE")
slurm_mem    = os.getenv("SLURM_MEM_PER_NODE") or os.getenv("SLURM_MEM_PER_CPU")

# Disk space in results_dir
os.makedirs(results_dir, exist_ok=True)
disk = shutil.disk_usage(results_dir)
disk_free = disk.free; disk_total = disk.total

# Reference and reads sizes
ref_size = os.path.getsize(REFERENCE) if REFERENCE and os.path.exists(REFERENCE) else None
reads_total, reads_detail = list_file_sizes(READS) if READS else (0, [])

# Samples count
n_fastq = len(READS) if READS else 0
n_pairs = len(PAIRS) if PAIRS else 0

# -------------------------
# Heuristic guidance
# -------------------------
problems = []
notes = []

# Memory heuristics (ballpark):
# - bwa-mem/mem2: ~1–2 GB per thread is comfortable for big genomes; spike is modest.
# - samtools sort: needs ~ (input BAM size) extra temp; rule of thumb: reserve >= 8–16 GB.
# - GATK often benefits from >16 GB; bcftools can work with less.
# We don't know BAM size yet; estimate from compressed reads.
est_genome_gb = None
if REFERENCE and ref_size:
    # Genome FASTA disk size isn't equal to genome size (depends on masking/compression),
    # but in practice: plain text FASTA bytes ~= number of bases + newlines.
    # We'll approximate genome size as ref_size bytes (lower bound).
    est_genome_gb = ref_size / (1024**3)

reads_gb = reads_total / (1024**3) if reads_total else 0.0

# Recommend threads so that mem_per_thread >= 1.5 GB
if mem_bytes:
    mem_gb = mem_bytes / (1024**3)
    recommended_threads_by_mem = max(1, int(mem_gb // 1.5))
    recommended_threads = max(1, min(cpu_count, recommended_threads_by_mem))
else:
    recommended_threads = max(1, cpu_count // 2)  # conservative fallback

# Flag low memory relative to big genomes
if mem_bytes:
    mem_gb = mem_bytes / (1024**3)
    if est_genome_gb and est_genome_gb >= 2.0 and mem_gb < 32:
        problems.append("Memory < 32 GB for a large genome; expect slow or failed sorting/calling.")
    if reads_gb and reads_gb > 10 and mem_gb < 32:
        problems.append("Many/large reads with < 32 GB RAM; consider fewer threads or chunking.")
else:
    notes.append("Could not determine memory limit; cgroup/Slurm not detected.")

# Disk space warning
if disk_free < 50 * (1024**3):
    problems.append("Less than 50 GB free in results directory; sorting and VCF steps may run out of space.")
elif disk_free < 20 * (1024**3):
    problems.append("Less than 20 GB free in results directory; very likely to fail during sort or VCF writing.")

# Pairing expectation
if n_fastq > 0 and n_pairs == 0:
    problems.append("No R1/R2 pairs detected; pipeline expects paired-end data.")

# -------------------------
# Render compact summary
# -------------------------
def row(label, value):
    return f"<div><b>{label}:</b> {value}</div>"

def code(s):
    return f"<code>{s}</code>"

def nice_bool(b):
    return "Yes" if b else "No"

html = []
html.append(f"<div style='border:1px solid #444; padding:8px; border-radius:6px; font-size:14px; line-height:1.35;'>")
html.append("<div style='font-weight:600; margin-bottom:6px;'>Compute environment</div>")

# CPU/Mem
html.append(row("CPUs visible", cpu_count))
html.append(row("Memory limit", mem_str))
if slurm_jobid:
    html.append(row("Slurm job", code(slurm_jobid)))
    if slurm_nodel: html.append(row("Node(s)", code(slurm_nodel)))
    if slurm_cpt:   html.append(row("SLURM_CPUS_PER_TASK", code(slurm_cpt)))
    if slurm_con:   html.append(row("SLURM_CPUS_ON_NODE", code(slurm_con)))
    if slurm_mem:   html.append(row("SLURM_MEM_*", code(slurm_mem)))

# Disk
html.append(row("Results dir", code(results_dir)))
html.append(row("Disk free / total", f"{human_bytes(disk_free)} / {human_bytes(disk_total)}"))

# Data summary
html.append("<hr style='margin:6px 0;'>")
html.append("<div style='font-weight:600; margin-bottom:4px;'>Data summary</div>")
html.append(row("Reference", code(REFERENCE if REFERENCE else 'None')))
html.append(row("Ref size (bytes)", f"{ref_size:,}" if ref_size is not None else "Unknown"))
html.append(row("Read dir", code(READ_DIR if READ_DIR else 'None')))
html.append(row("FASTQ files", n_fastq))
html.append(row("Samples (pairs)", n_pairs))
html.append(row("Total reads size", f"{reads_gb:.2f} GB"))

# Recommendations
html.append("<hr style='margin:6px 0;'>")
html.append("<div style='font-weight:600; margin-bottom:4px;'>Recommendations</div>")
html.append(row("Threads suggestion", f"{recommended_threads} (limit threads so ~≥1.5 GB RAM per thread)"))
html.append("<div style='margin-top:4px; font-size:13px;'>"
            "• For <b>bwa</b>, set <code>-t {recommended_threads}</code>.<br>"
            "• For <b>samtools sort</b>, use <code>-@ {recommended_threads}</code> and tune <code>-m</code> (e.g., 1G–2G).<br>"
            "• For <b>bcftools</b>, modest threading helps; <b>GATK</b> benefits from larger RAM (&ge;16–32 GB).</div>".format(recommended_threads=recommended_threads))

# Problems / Notes
if problems:
    html.append("<hr style='margin:6px 0;'><div style='color:#b71c1c; font-weight:600;'>Warnings</div>")
    html.append("<ul style='margin:4px 0; padding-left:18px;'>" + "".join([f"<li>{p}</li>" for p in problems]) + "</ul>")
else:
    html.append("<p style='color:#2e7d32; margin:6px 0;'><b>All good.</b> Resources look reasonable for a test run.</p>")

if notes:
    html.append("<div style='margin-top:6px; color:#555;'><b>Notes:</b> " + " ".join(notes) + "</div>")

html.append("</div>")
display(HTML("".join(html)))

# Expose a variable for downstream cells
CPU_THREADS_SUGGESTED = recommended_threads

## 2. Doing the work:

In [4]:
## This cell will make sure everything is in the right place
import os

# Expect these from the selection cell:
# - PROJECT_ROOT
# - selected_ref (path to reference FASTA)
# - read_dir_dropdown.value (read set name)
READ_SET_NAME = read_dir_dropdown.value  # e.g., "Brachycybe_producta_test_readset"
REFERENCE_SRC = selected_ref

# Create a run-specific area for reference indices
RUN_DIR = f"{PROJECT_ROOT}/results/{READ_SET_NAME}"
REF_WORKDIR = f"{RUN_DIR}/reference"
os.makedirs(REF_WORKDIR, exist_ok=True)

# Place a symlink to the reference FASTA inside the run reference dir
ref_basename = os.path.basename(REFERENCE_SRC)
REF_FASTA = os.path.join(REF_WORKDIR, ref_basename)

# Create/update symlink (safe to re-run)
if os.path.islink(REF_FASTA) or os.path.exists(REF_FASTA):
    # If it's a dangling or wrong symlink, remove and relink
    if os.path.islink(REF_FASTA) and os.readlink(REF_FASTA) != REFERENCE_SRC:
        os.remove(REF_FASTA)
        os.symlink(REFERENCE_SRC, REF_FASTA)
else:
    os.symlink(REFERENCE_SRC, REF_FASTA)

print("Reference workdir:", REF_WORKDIR)
print("Reference FASTA (symlink in run dir):", REF_FASTA)

# Export to environment so bash cells can use $REF_FASTA
os.environ["REF_FASTA"] = REF_FASTA

Reference workdir: /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/reference
Reference FASTA (symlink in run dir): /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/reference/GCA_036925085.1_qdBraProd1.0.pri_genomic.fa


## 3. Launch snakemake!!!

In [7]:
# === Snakemake launcher cell ============================================
# Run the Snakemake workflow from inside the Jupyter notebook.
#
# Requirements:
#  - workflow/Snakefile exists (we'll build it next)
#  - profiles/slurm/ contains Snakemake profile files (optional)
#  - PAIRS, READ_SET_NAME, REF_FASTA exist (from earlier cells)
#
# Modes:
#   mode = "local"  → single-node run (good for tiny test dataset)
#   mode = "slurm"  → submit jobs to Slurm using your profile
#
# =======================================================================

import os, sys, subprocess
from pathlib import Path
from IPython.display import display, Markdown

# --- Select mode ---
mode = "local"        # <-- change to "slurm" when ready
#mode = "slurm"

# --- Safety checks ---
if "PAIRS" not in globals() or len(PAIRS) == 0:
    raise RuntimeError("PAIRS dict is empty. Run your read‑selection cell first.")

if "REF_FASTA" not in globals():
    raise RuntimeError("REF_FASTA not defined. Run reference‑indexing cell first.")

# --- Directory setup ---
PROJECT_ROOT = Path(PROJECT_ROOT)
print("project_root = ", PROJECT_ROOT)
READ_SET_NAME = READ_SET_NAME if "READ_SET_NAME" in globals() else read_dir_dropdown.value
RUN_DIR = PROJECT_ROOT / "results" / READ_SET_NAME
CFG_DIR = PROJECT_ROOT / "workflow" / "config"
CFG_DIR.mkdir(parents=True, exist_ok=True)
SNAKEFILE = PROJECT_ROOT / "Snakefile"

# --- Write samples.tsv from PAIRS ---
samples_tsv = CFG_DIR / "samples.tsv"
with open(samples_tsv, "w") as f:
    f.write("sample\tR1\tR2\n")
    for sid, p in sorted(PAIRS.items()):
        f.write(f"{sid}\t{p['R1']}\t{p['R2']}\n")

# --- Write config.yaml ---
config_yaml = CFG_DIR / "config.yaml"
config_yaml.write_text(
    f"""
project_root: {PROJECT_ROOT}
read_set: {READ_SET_NAME}
reference: {Path(REF_FASTA).name}      # basename only; Snakefile resolves path
caller: bcftools                       # or "gatk"
threads_per_sample: 16
mem_gb_per_sample: 64
"""
)

display(Markdown(f"### 📝 Wrote config files:\n- `{samples_tsv}`\n- `{config_yaml}`"))

# --- Snakemake command ---
snk = [
    "snakemake",
    "--directory", str(PROJECT_ROOT),
    "-s", str(SNAKEFILE),
    "--cores", "1",                         # local scheduler cores
    "--printshellcmds",
    #"--rerun-incomplete",
    "--forceall",                          # for debug
]

if mode == "slurm":
    snk += [
        "--profile", "/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/profiles/slurm",
        "--jobs", "80",
    ]
else:
    # local mode for tiny tests
    snk += ["--cores", str(CPU_THREADS_SUGGESTED)]

display(Markdown(f"### 🚀 Running Snakemake in **{mode}** mode...\n```\n{' '.join(snk)}\n```"))

# --- Run Snakemake and stream output ---
process = subprocess.Popen(snk, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in process.stdout:
    print(line, end="")

process.wait()

if process.returncode == 0:
    display(Markdown("### 🎉 Snakemake pipeline complete!"))
else:
    raise RuntimeError(f"Snakemake failed with exit code {process.returncode}.")


project_root =  /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline


### 📝 Wrote config files:
- `/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/workflow/config/samples.tsv`
- `/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/workflow/config/config.yaml`

### 🚀 Running Snakemake in **local** mode...
```
snakemake --directory /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline -s /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/Snakefile --cores 1 --printshellcmds --forceall --cores 64
```

Assuming unrestricted shared filesystem usage.
host: gpu-10-58
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 64
Rules claiming more threads will be scaled down.
Job stats:
job                  count
-----------------  -------
align_sort               2
all                      1
bcftools_call            1
dict                     1
faidx                    1
markdup_to_cram          2
symlink_reference        1
trim_reads               2
total                   11

Select jobs to execute...
Execute 3 jobs...

[Wed Jan 21 15:35:01 2026]
localrule trim_reads:
    input: /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/data/read_directories/Brachycybe_producta_test_readset/CCGPMC021_BMEA101907_S2_L004_R1_001.fastq.gz, /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/data/read_directories/Brachycybe_producta_test_readset/CCGPMC021_BMEA101907_S2_L004_R2_001.fastq.gz
    output: results/Brachycybe_producta_test_readset/trimmed/CCGPMC

RuntimeError: Snakemake failed with exit code 1.

In [ ]:
!snakemake --directory /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/ \
-s /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/Snakefile \
--dry-run -p

In [8]:

print("project_root =", globals().get("project_root"))
print("PROJECT_ROOT =", globals().get("PROJECT_ROOT"))

import os
print("Does workflow/Snakefile exist?", os.path.exists(os.path.join(PROJECT_ROOT, "workflow", "Snakefile")))


project_root = None
PROJECT_ROOT = /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline
Does workflow/Snakefile exist? False


In [9]:
!pwd

/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/notebooks


In [10]:
!snakemake --directory /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline -s /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/Snakefile --cores 1 --dry-run -p

host: gpu-10-58
Building DAG of jobs...
Job stats:
job                count
---------------  -------
align_sort             2
all                    1
bcftools_call          1
markdup_to_cram        2
total                  6


[Wed Jan 21 15:36:36 2026]
rule align_sort:
    input: results/Brachycybe_producta_test_readset/trimmed/CCGPMC021_BMEA101904_S1_L004001.R1.trimmed.fastq.gz, results/Brachycybe_producta_test_readset/trimmed/CCGPMC021_BMEA101904_S1_L004001.R2.trimmed.fastq.gz, results/Brachycybe_producta_test_readset/reference/GCA_036925085.1_qdBraProd1.0.pri_genomic.fa, results/Brachycybe_producta_test_readset/reference/GCA_036925085.1_qdBraProd1.0.pri_genomic.fa.fai
    output: results/Brachycybe_producta_test_readset/alignments/CCGPMC021_BMEA101904_S1_L004001.sorted.bam
    jobid: 5
    reason: Missing output files: results/Brachycybe_producta_test_readset/alignments/CCGPMC021_BMEA101904_S1_L004001.sorted.bam
    wildcards: sample=CCGPMC021_BMEA101904_S1_L004001
    resources: 

In [11]:
## INDEX the reference files (If not already done)
## This takes a few minutes to run.
import os
import shutil
import subprocess
from pathlib import Path

# ---- Inputs expected from earlier cells ----
try:
    PROJECT_ROOT
    selected_ref
    read_dir_dropdown  # to get the read set name
except NameError as e:
    raise RuntimeError(
        "Required variables are missing. Please run the selection/setup cell first."
    ) from e

READ_SET_NAME = read_dir_dropdown.value
REFERENCE_SRC = selected_ref

# ---- Directories for this run ----
RUN_DIR = Path(PROJECT_ROOT) / "results" / READ_SET_NAME
REF_WORKDIR = RUN_DIR / "reference"
REF_WORKDIR.mkdir(parents=True, exist_ok=True)

# ---- Symlink the reference into the run dir ----
ref_basename = os.path.basename(REFERENCE_SRC)
REF_FASTA = REF_WORKDIR / ref_basename

def ensure_symlink(src: Path | str, dest: Path):
    """Create/refresh a symlink (dest) pointing to src."""
    src = Path(src)
    if dest.exists() or dest.is_symlink():
        # If it's already a symlink to the correct target, keep it
        if dest.is_symlink() and os.path.realpath(dest) == os.path.realpath(src):
            return
        # Otherwise remove and replace
        dest.unlink()
    dest.symlink_to(src)

ensure_symlink(REFERENCE_SRC, REF_FASTA)

print(f"Reference workdir: {REF_WORKDIR}")
print(f"Reference FASTA (symlink): {REF_FASTA}")

# ---- Helpers to run commands and check availability ----
def have_tool(name: str) -> bool:
    return shutil.which(name) is not None

def run_cmd(cmd: list[str], check: bool = True):
    print("  $", " ".join(cmd))
    try:
        result = subprocess.run(cmd, check=check, capture_output=True, text=True)
        if result.stdout:
            print(result.stdout.strip())
        if result.stderr:
            # Many bio tools write progress to stderr; print but don’t treat as error unless check fails
            print(result.stderr.strip())
    except subprocess.CalledProcessError as e:
        print(e.stdout or "")
        print(e.stderr or "")
        raise

# ---- Compute expected index files ----
fasta_path = str(REF_FASTA)
fai_path   = f"{fasta_path}.fai"
# GATK dict convention: same stem, .dict extension (replace only last extension)
stem_no_ext = str(REF_FASTA).rsplit(".", 1)[0]
dict_path  = f"{stem_no_ext}.dict"

# BWA vs BWA-MEM2 index file sets
bwa_mem2_files = [f"{fasta_path}.0123", f"{fasta_path}.pac"]
bwa_files      = [f"{fasta_path}.bwt", f"{fasta_path}.sa"]

# ---- 1) samtools faidx ----
print("\n[1/3] samtools faidx")
if not have_tool("samtools"):
    raise RuntimeError(
        "samtools not found in PATH. Activate the conda env (e.g., Bondlab_phylo_env) or install samtools."
    )

if not os.path.exists(fai_path):
    print("  Creating FASTA index (.fai)")
    run_cmd(["samtools", "faidx", fasta_path])
else:
    print("  .fai already exists, skipping.")

# ---- 2) GATK CreateSequenceDictionary ----
print("\n[2/3] GATK CreateSequenceDictionary")
if not os.path.exists(dict_path):
    if have_tool("gatk"):
        print("  Creating sequence dictionary (.dict)")
        run_cmd(["gatk", "CreateSequenceDictionary", "-R", fasta_path, "-O", dict_path])
    else:
        # Optional: Try Picard if available
        if have_tool("picard"):
            print("  GATK not found. Using Picard to create .dict")
            run_cmd(["picard", "CreateSequenceDictionary", f"R={fasta_path}", f"O={dict_path}"])
        else:
            raise RuntimeError(
                "Neither GATK nor Picard found. Install one of them to create the sequence dictionary."
            )
else:
    print("  .dict already exists, skipping.")

# ---- 3) BWA or BWA-MEM2 index ----
print("\n[3/3] BWA/BWA-MEM2 index")
if have_tool("bwa-mem2"):
    # Check presence of mem2 index artifacts
    if not all(os.path.exists(p) for p in bwa_mem2_files):
        print("  Building bwa-mem2 index")
        run_cmd(["bwa-mem2", "index", fasta_path])
    else:
        print("  bwa-mem2 index already present, skipping.")
elif have_tool("bwa"):
    if not all(os.path.exists(p) for p in bwa_files):
        print("  Building classic bwa index")
        run_cmd(["bwa", "index", fasta_path])
    else:
        print("  bwa index already present, skipping.")
else:
    raise RuntimeError(
        "Neither bwa-mem2 nor bwa found in PATH. Install one to proceed with alignment."
    )

print("\nDone indexing.")
print(f"Generated/validated:\n  {fai_path}\n  {dict_path}\n  (BWA index files alongside {REF_FASTA})")

Reference workdir: /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/reference
Reference FASTA (symlink): /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/reference/GCA_036925085.1_qdBraProd1.0.pri_genomic.fa

[1/3] samtools faidx
  .fai already exists, skipping.

[2/3] GATK CreateSequenceDictionary
  .dict already exists, skipping.

[3/3] BWA/BWA-MEM2 index
  Building bwa-mem2 index
  $ bwa-mem2 index /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/reference/GCA_036925085.1_qdBraProd1.0.pri_genomic.fa
ref_seq_len = 590789254
count = 0, 191829304, 295394627, 398959950, 590789254
BWT[408233426] = 4
CP_SHIFT = 6, CP_MASK = 63
sizeof CP_OCC = 64
max_occ_ind = 9231082
Looking to launch executable "/group/jbondgrp2/stephenRichards/_conda_envs/Bondlab_phylo_env/bin/bwa-mem2.avx512bw", simd = .avx512bw
Launchi

In [12]:
## This is the qc cell - fastQC + Trimming (Cutadapt or Trimmomatic)
import os, shutil, subprocess
from pathlib import Path

# ---------- Inputs expected ----------
try:
    PROJECT_ROOT
    READ_SET_NAME
except NameError:
    # derive READ_SET_NAME from widget if needed
    READ_SET_NAME = read_dir_dropdown.value

# Expected from selection cell:
#   PAIRS: dict sample_id -> {'R1': path, 'R2': path}
#   FORMAT: 'fastq' or 'fastq.gz'
# Optional from compute check:
#   CPU_THREADS_SUGGESTED
if 'CPU_THREADS_SUGGESTED' not in globals() or CPU_THREADS_SUGGESTED is None:
    CPU_THREADS_SUGGESTED = max(1, (os.cpu_count() or 2) // 2)

# Tool preference (auto picks cutadapt if available)
TRIMMER = "auto"   # options: "auto", "cutadapt", "trimmomatic"

# ---------- Paths ----------
RUN_DIR = Path(PROJECT_ROOT) / "results" / READ_SET_NAME
QC_RAW_DIR = RUN_DIR / "qc" / "fastqc_raw"
QC_TRIM_DIR = RUN_DIR / "qc" / "fastqc_trimmed"
TRIM_DIR = RUN_DIR / "trimmed"
ADAPTERS_DIR = Path(PROJECT_ROOT) / "data" / "adapters"
ADAPTERS_FILE = ADAPTERS_DIR / "adapters.fasta"

for d in [QC_RAW_DIR, QC_TRIM_DIR, TRIM_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ---------- Utilities ----------
def have_tool(name: str) -> bool:
    return shutil.which(name) is not None

def run_cmd(cmd: list[str], cwd=None):
    print("  $", " ".join(cmd))
    res = subprocess.run(cmd, cwd=cwd, capture_output=True, text=True)
    # FastQC/Trimmers are chatty on stderr; show both
    if res.stdout.strip():
        print(res.stdout.strip())
    if res.stderr.strip():
        print(res.stderr.strip())
    if res.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(cmd)}")

def infer_ext():
    # Determine extension for trimmed outputs (preserve .gz if input is gzipped)
    if FORMAT is None:
        # Best-effort guess: check an R1 file
        first = next(iter(PAIRS.values()))
        gz = first['R1'].lower().endswith(".gz")
        return ".fastq.gz" if gz else ".fastq"
    return ".fastq.gz" if "gz" in FORMAT else ".fastq"

TRIM_EXT = infer_ext()

# Choose trimmer
def pick_trimmer():
    if TRIMMER == "cutadapt":
        if not have_tool("cutadapt"):
            raise RuntimeError("TRIMMER='cutadapt' but cutadapt not found in PATH.")
        return "cutadapt"
    if TRIMMER == "trimmomatic":
        if not have_tool("trimmomatic"):
            raise RuntimeError("TRIMMER='trimmomatic' but trimmomatic not found in PATH.")
        return "trimmomatic"
    # auto
    if have_tool("cutadapt"):
        return "cutadapt"
    if have_tool("trimmomatic"):
        return "trimmomatic"
    raise RuntimeError("No trimmer found. Install cutadapt or trimmomatic in your conda env.")

TRIMMER_USED = pick_trimmer()
print(f"Using trimmer: {TRIMMER_USED}")

# Check FastQC
if not have_tool("fastqc"):
    raise RuntimeError("FastQC not found. Please install fastqc in your conda environment.")

# Check adapters file (recommended but not strictly required for cutadapt if using built-in adapters)
if not ADAPTERS_FILE.exists():
    print(f"WARNING: Adapters file not found at {ADAPTERS_FILE}. Proceeding without explicit adapters.")

# ---------- Per-sample QC workflow ----------
summary = []
for sample_id, paths in sorted(PAIRS.items()):
    r1 = paths["R1"]
    r2 = paths["R2"]

    # FastQC (raw)
    print(f"\n[FastQC: raw] {sample_id}")
    # FastQC writes a .html and .zip per input; it auto-detects gzip
    # To be idempotent, check if expected outputs exist
    r1_html = QC_RAW_DIR / (Path(r1).name.replace(".gz", "") + "_fastqc.html")
    r2_html = QC_RAW_DIR / (Path(r2).name.replace(".gz", "") + "_fastqc.html")

    need_raw_fastqc = not (r1_html.exists() and r2_html.exists())
    if need_raw_fastqc:
        run_cmd(["fastqc", "-t", str(max(1, min(CPU_THREADS_SUGGESTED, 8))), "-o", str(QC_RAW_DIR), r1, r2])
    else:
        print("  Raw FastQC outputs already exist, skipping.")

    # Trimming
    trimmed_r1 = TRIM_DIR / f"{sample_id}.R1.trimmed{TRIM_EXT}"
    trimmed_r2 = TRIM_DIR / f"{sample_id}.R2.trimmed{TRIM_EXT}"
    unp_r1 = TRIM_DIR / f"{sample_id}.R1.unpaired{TRIM_EXT.replace('.gz','')}"
    unp_r2 = TRIM_DIR / f"{sample_id}.R2.unpaired{TRIM_EXT.replace('.gz','')}"

    if trimmed_r1.exists() and trimmed_r2.exists():
        print(f"[Trim] {sample_id} — trimmed files exist, skipping.")
    else:
        print(f"[Trim] {sample_id} — running {TRIMMER_USED}")
        if TRIMMER_USED == "cutadapt":
            # Cutadapt paired-end: remove adapters from both ends, quality trim, min length
            # If adapters file present, use -g/-a with file; otherwise use common Illumina presets via -a/-A
            cmd = [
                "cutadapt",
                "-j", str(CPU_THREADS_SUGGESTED),
                "-q", "15,15",            # quality trim both ends
                "-m", "36",               # minimum length
                "-o", str(trimmed_r1),
                "-p", str(trimmed_r2),
            ]
            if ADAPTERS_FILE.exists():
                # Use file as 5' and 3' adapters for both reads (cutadapt accepts fasta of adapters)
                # -g / -G for 5' (front), -a / -A for 3' (back); using as generic adapters
                cmd += ["-g", f"file:{ADAPTERS_FILE}", "-G", f"file:{ADAPTERS_FILE}",
                        "-a", f"file:{ADAPTERS_FILE}", "-A", f"file:{ADAPTERS_FILE}"]
            else:
                # fallback: common Illumina universal adapters
                cmd += ["-a", "AGATCGGAAGAGC", "-A", "AGATCGGAAGAGC"]
            cmd += [r1, r2]
            run_cmd(cmd)

        elif TRIMMER_USED == "trimmomatic":
            # Trimmomatic PE mode; assumes adapter fasta is compatible
            # Typical settings comparable to what you pasted earlier
            # Note: trimmomatic name can be "trimmomatic" or "trimmomatic.jar" with "java -jar"
            # On conda, "trimmomatic" wrapper is available.
            threads = str(max(1, CPU_THREADS_SUGGESTED))
            # Pick appropriate phred auto-detect; modern data = phred33
            cmd = [
                "trimmomatic", "PE",
                "-threads", threads,
                r1, r2,
                str(trimmed_r1), str(unp_r1),
                str(trimmed_r2), str(unp_r2),
                f"ILLUMINACLIP:{ADAPTERS_FILE if ADAPTERS_FILE.exists() else 'TruSeq3-PE.fa'}:2:30:10",
                "LEADING:3", "TRAILING:3", "SLIDINGWINDOW:4:15", "MINLEN:36"
            ]
            run_cmd(cmd)

    # FastQC on trimmed reads
    print(f"[FastQC: trimmed] {sample_id}")
    t1_html = QC_TRIM_DIR / (trimmed_r1.name.replace(".gz", "") + "_fastqc.html")
    t2_html = QC_TRIM_DIR / (trimmed_r2.name.replace(".gz", "") + "_fastqc.html")
    need_trim_fastqc = not (t1_html.exists() and t2_html.exists())
    if need_trim_fastqc:
        run_cmd(["fastqc", "-t", str(max(1, min(CPU_THREADS_SUGGESTED, 8))), "-o", str(QC_TRIM_DIR),
                 str(trimmed_r1), str(trimmed_r2)])
    else:
        print("  Trimmed FastQC outputs already exist, skipping.")

    summary.append({
        "sample": sample_id,
        "raw_fastqc": "ok" if not need_raw_fastqc else "ran",
        "trim_tool": TRIMMER_USED,
        "trimmed_r1": str(trimmed_r1),
        "trimmed_r2": str(trimmed_r2),
        "trim_fastqc": "ok" if not need_trim_fastqc else "ran",
    })

print("\nQC summary:")
for row in summary:
    print(f"  {row['sample']}: trim={row['trim_tool']}, trimmed=({Path(row['trimmed_r1']).name}, {Path(row['trimmed_r2']).name})")
print("\nOutputs:")
print(f"  Raw FastQC:     {QC_RAW_DIR}")
print(f"  Trimmed reads:  {TRIM_DIR}")
print(f"  Trimmed FastQC: {QC_TRIM_DIR}")


Using trimmer: cutadapt

[FastQC: raw] CCGPMC021_BMEA101904_S1_L004001
  $ fastqc -t 8 -o /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/qc/fastqc_raw /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/data/read_directories/Brachycybe_producta_test_readset/CCGPMC021_BMEA101904_S1_L004_R1_001.fastq.gz /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/data/read_directories/Brachycybe_producta_test_readset/CCGPMC021_BMEA101904_S1_L004_R2_001.fastq.gz
application/gzip
application/gzip
Analysis complete for CCGPMC021_BMEA101904_S1_L004_R1_001.fastq.gz
Analysis complete for CCGPMC021_BMEA101904_S1_L004_R2_001.fastq.gz
Started analysis of CCGPMC021_BMEA101904_S1_L004_R1_001.fastq.gz
Started analysis of CCGPMC021_BMEA101904_S1_L004_R2_001.fastq.gz
Approx 5% complete for CCGPMC021_BMEA101904_S1_L004_R1_001.fastq.gz
Approx 5% complete for CCGPMC021_BMEA101904_S1_L004_R2_001.fastq.gz
Approx 10% 

In [ ]:
# 3. Align reads to the reference
# Align with BWA-MEM
#bwa mem -t 8 reference.fasta sample_R1.trimmed.fastq.gz sample_R2.trimmed.fastq.gz > sample.sam

In [ ]:
# 4. Convert SAM → BAM, Sort, and Index
#samtools view -bS sample.sam | samtools sort -o sample.sorted.bam
#samtools index sample.sorted.bam

In [ ]:
# 5. Mark Duplicates
#gatk MarkDuplicates -I sample.sorted.bam -O sample.dedup.bam -M sample.metrics.txt
#samtools index sample.dedup.bam

In [13]:
## Alignment → Sort → Dedup → Index (Python)
import os, shutil, subprocess
from pathlib import Path

# ----------- Inputs from earlier cells -----------
try:
    PROJECT_ROOT
except NameError:
    raise RuntimeError("PROJECT_ROOT not set. Run the selection/setup cells first.")

# From selection cell
READ_SET_NAME = globals().get("READ_SET_NAME", None) or read_dir_dropdown.value
PAIRS         = globals().get("PAIRS", {})
# From indexing cell (symlinked FASTA in results/<read_set>/reference)
try:
    REF_FASTA
except NameError:
    raise RuntimeError("REF_FASTA not set. Run the indexing cell before alignment.")

# From compute cell (optional). Fall back to half of visible CPUs if not set.
CPU_THREADS_SUGGESTED = globals().get("CPU_THREADS_SUGGESTED", None) or max(1, (os.cpu_count() or 2)//2)

RUN_DIR    = Path(PROJECT_ROOT) / "results" / READ_SET_NAME
TRIM_DIR   = RUN_DIR / "trimmed"
ALN_DIR    = RUN_DIR / "alignments"
LOGS_DIR   = RUN_DIR / "logs"
for d in (ALN_DIR, LOGS_DIR):
    d.mkdir(parents=True, exist_ok=True)

# ----------- Helpers -----------
def have_tool(name: str) -> bool:
    return shutil.which(name) is not None

def run_cmd(cmd: list[str], log_path: Path | None = None, cwd=None):
    """Run a command, capture stdout/stderr to screen and optional log file."""
    print("  $", " ".join(cmd))
    proc = subprocess.Popen(cmd, cwd=cwd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    out, err = proc.communicate()
    if log_path:
        with open(log_path, "w") as lf:
            lf.write("CMD: " + " ".join(cmd) + "\n\n")
            lf.write("STDOUT:\n" + (out or "") + "\n\n")
            lf.write("STDERR:\n" + (err or "") + "\n")
    if out.strip():
        print(out.strip())
    if err.strip():
        print(err.strip())
    if proc.returncode != 0:
        raise RuntimeError(f"Command failed ({proc.returncode}): {' '.join(cmd)}")

def pick_aligner():
    # Prefer bwa-mem2
    if have_tool("bwa-mem2"):
        return "bwa-mem2"
    if have_tool("bwa"):
        return "bwa"
    raise RuntimeError("No aligner found. Install bwa-mem2 (preferred) or bwa in your conda env.")

ALIGNER = pick_aligner()
print(f"Aligner: {ALIGNER}")
print(f"Threads: {CPU_THREADS_SUGGESTED}")

# Determine whether trimmed files are gzipped to set output extension consistently
def trimmed_ext():
    # Inspect one existing trimmed file if present, else default to .fastq.gz
    for sid, paths in sorted(PAIRS.items()):
        candidate = TRIM_DIR / f"{sid}.R1.trimmed.fastq.gz"
        if candidate.exists():
            return ".fastq.gz"
        candidate = TRIM_DIR / f"{sid}.R1.trimmed.fastq"
        if candidate.exists():
            return ".fastq"
    return ".fastq.gz"  # default
TRIM_EXT = trimmed_ext()

# ----------- Pipeline per sample -----------
completed = []
for sample_id, paths in sorted(PAIRS.items()):
    # Expect trimmed files from the QC step
    r1 = TRIM_DIR / f"{sample_id}.R1.trimmed{TRIM_EXT}"
    r2 = TRIM_DIR / f"{sample_id}.R2.trimmed{TRIM_EXT}"
    if not r1.exists() or not r2.exists():
        print(f"[{sample_id}] WARNING: Trimmed reads not found for sample (expected {r1.name} and {r2.name}). Skipping.")
        continue

    # File paths
    bam_aln   = ALN_DIR / f"{sample_id}.aligned.bam"
    bam_sorted= ALN_DIR / f"{sample_id}.sorted.bam"
    bam_dedup = ALN_DIR / f"{sample_id}.dedup.bam"
    bai_dedup = ALN_DIR / f"{sample_id}.dedup.bam.bai"
    log_align = LOGS_DIR / f"{sample_id}.align.log"
    log_sort  = LOGS_DIR / f"{sample_id}.sort.log"
    log_md    = LOGS_DIR / f"{sample_id}.markdup.log"

    print(f"\n=== Sample: {sample_id} ===")

    # Read group (RG) fields — adjust as needed for your lab conventions
    # ID: unique per sample/run, SM: sample name, PL: platform, LB: library (use read set), PU: platform unit (lane info if present)
    # We'll derive PU heuristically from filenames; safe to keep simple if not meaningful.
    RG_ID = sample_id
    RG_SM = sample_id
    RG_PL = "ILLUMINA"
    RG_LB = READ_SET_NAME
    # Attempt to extract a lane token like 'L004' or similar from filenames
    base_r1 = Path(paths["R1"]).name
    lane_token = None
    for tok in base_r1.split("_"):
        if tok.upper().startswith("L") and tok[1:].isdigit():
            lane_token = tok
            break
    RG_PU = f"{sample_id}.{lane_token}" if lane_token else sample_id

    rg_string = f"@RG\\tID:{RG_ID}\\tSM:{RG_SM}\\tPL:{RG_PL}\\tLB:{RG_LB}\\tPU:{RG_PU}"

    # 1) Align → BAM (pipe through samtools view)
    if not bam_aln.exists():
        print(f"[{sample_id}] Aligning with {ALIGNER}...")
        if ALIGNER == "bwa-mem2":
            cmd = [
                "bwa-mem2", "mem",
                "-t", str(CPU_THREADS_SUGGESTED),
                "-R", rg_string,
                str(REF_FASTA),
                str(r1), str(r2)
            ]
        else:  # "bwa"
            cmd = [
                "bwa", "mem",
                "-t", str(CPU_THREADS_SUGGESTED),
                "-R", rg_string,
                str(REF_FASTA),
                str(r1), str(r2)
            ]
        # Pipe to samtools view to get BAM directly
        # Using shell pipeline is messy in pure Python; we can run bwa then samtools in a pipe with Popen
        print("  Piping to samtools view (BAM)...")
        with open(bam_aln, "wb") as bam_out:
            bwa_proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=False)
            view_proc = subprocess.Popen(["samtools", "view", "-bS", "-"],
                                         stdin=bwa_proc.stdout, stdout=bam_out, stderr=subprocess.PIPE, text=False)
            bwa_proc.stdout.close()
            _, bwa_err = bwa_proc.communicate()
            view_out, view_err = view_proc.communicate()
            # Log stderr streams
            with open(log_align, "w") as lf:
                lf.write("CMD: " + " ".join(cmd) + " | samtools view -bS -\n\n")
                lf.write("BWA/ALIGN STDERR:\n" + (bwa_err.decode('utf-8', errors='ignore') if isinstance(bwa_err, (bytes, bytearray)) else str(bwa_err)) + "\n\n")
                lf.write("SAMTOOLS VIEW STDERR:\n" + (view_err.decode('utf-8', errors='ignore') if isinstance(view_err, (bytes, bytearray)) else str(view_err)) + "\n")
            if bwa_proc.returncode != 0 or view_proc.returncode != 0:
                raise RuntimeError(f"[{sample_id}] Alignment failed. See log: {log_align}")
    else:
        print(f"[{sample_id}] Aligned BAM exists, skipping alignment.")

    # 2) Sort BAM
    if not bam_sorted.exists():
        print(f"[{sample_id}] Sorting BAM...")
        run_cmd(["samtools", "sort",
                 "-@", str(CPU_THREADS_SUGGESTED),
                 "-o", str(bam_sorted),
                 str(bam_aln)], log_path=log_sort)
    else:
        print(f"[{sample_id}] Sorted BAM exists, skipping sort.")

    # 3) Mark duplicates (GATK preferred; fallback to picard)
    if not bam_dedup.exists():
        print(f"[{sample_id}] Marking duplicates...")
        metrics = ALN_DIR / f"{sample_id}.markdup.metrics.txt"
        if have_tool("gatk"):
            run_cmd(["gatk", "MarkDuplicates",
                     "-I", str(bam_sorted),
                     "-O", str(bam_dedup),
                     "-M", str(metrics),
                     "--CREATE_INDEX", "true"], log_path=log_md)
        elif have_tool("picard"):
            run_cmd(["picard", "MarkDuplicates",
                     f"I={bam_sorted}", f"O={bam_dedup}", f"M={metrics}", "CREATE_INDEX=true"], log_path=log_md)
        else:
            raise RuntimeError("Neither GATK nor Picard found for MarkDuplicates.")
    else:
        print(f"[{sample_id}] Dedup BAM exists, skipping mark-duplicates.")

    # 4) Ensure index exists
    if not bai_dedup.exists():
        print(f"[{sample_id}] Indexing deduplicated BAM...")
        run_cmd(["samtools", "index", str(bam_dedup)])
    else:
        print(f"[{sample_id}] BAM index exists, skipping index.")

    completed.append(sample_id)

print("\nAlignment pipeline completed for samples:", ", ".join(completed) if completed else "(none)")
print("Outputs per sample are in:", ALN_DIR)


Aligner: bwa-mem2
Threads: 64

=== Sample: CCGPMC021_BMEA101904_S1_L004001 ===
[CCGPMC021_BMEA101904_S1_L004001] Aligning with bwa-mem2...
  Piping to samtools view (BAM)...
[CCGPMC021_BMEA101904_S1_L004001] Sorting BAM...
  $ samtools sort -@ 64 -o /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/alignments/CCGPMC021_BMEA101904_S1_L004001.sorted.bam /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/alignments/CCGPMC021_BMEA101904_S1_L004001.aligned.bam
[bam_sort_core] merging from 0 files and 64 in-memory blocks...
[CCGPMC021_BMEA101904_S1_L004001] Marking duplicates...
  $ gatk MarkDuplicates -I /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/alignments/CCGPMC021_BMEA101904_S1_L004001.sorted.bam -O /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_r

In [14]:
## Variant calling with bcftools (multisample)
import os, shutil, subprocess, glob
from pathlib import Path

# ----------- Inputs expected -----------
try:
    PROJECT_ROOT
except NameError:
    raise RuntimeError("PROJECT_ROOT not set. Run earlier setup cells.")

READ_SET_NAME = globals().get("READ_SET_NAME", None) or read_dir_dropdown.value
RUN_DIR    = Path(PROJECT_ROOT) / "results" / READ_SET_NAME
ALN_DIR    = RUN_DIR / "alignments"
VAR_DIR    = RUN_DIR / "variants" / "bcftools"
LOGS_DIR   = RUN_DIR / "logs"
for d in (VAR_DIR, LOGS_DIR):
    d.mkdir(parents=True, exist_ok=True)

# From indexing cell:
try:
    REF_FASTA
except NameError:
    raise RuntimeError("REF_FASTA is not defined. Run the indexing cell first.")

# Optional threads from compute cell
CPU_THREADS_SUGGESTED = globals().get("CPU_THREADS_SUGGESTED", None) or max(1, (os.cpu_count() or 2)//2)
THREADS = max(1, min(int(CPU_THREADS_SUGGESTED), 16))

def have_tool(name: str) -> bool:
    return shutil.which(name) is not None

def run_pipe_bcftools_mpileup_call(bams, out_vcf_gz, log_path):
    """Run: bcftools mpileup | bcftools call, writing bgzipped VCF."""
    mpileup_cmd = [
        "bcftools", "mpileup",
        "-Ou",              # uncompressed BCF to stdout
        "-f", str(REF_FASTA),
        "-q", "20",         # min MAPQ
        "-Q", "20",         # min base quality
        "-a", "FORMAT/DP,FORMAT/AD",  # carry per-sample depth/allele depth
    ] + [str(b) for b in bams]

    call_cmd = ["bcftools", "call", "-mv", "-Oz", "-o", str(out_vcf_gz)]

    print("  $", " ".join(mpileup_cmd), "|", " ".join(call_cmd))
    with open(log_path, "w") as lf:
        lf.write("CMD: " + " ".join(mpileup_cmd) + " | " + " ".join(call_cmd) + "\n\n")
        # Start mpileup
        p1 = subprocess.Popen(mpileup_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=False)
        # Pipe into call
        p2 = subprocess.Popen(call_cmd, stdin=p1.stdout, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=False)
        p1.stdout.close()

        out2, err2 = p2.communicate()
        out1, err1 = p1.communicate()

        lf.write("MPILEUP STDERR:\n" + (err1.decode('utf-8', errors='ignore') if isinstance(err1, (bytes, bytearray)) else str(err1)) + "\n\n")
        lf.write("CALL STDERR:\n"    + (err2.decode('utf-8', errors='ignore') if isinstance(err2, (bytes, bytearray)) else str(err2)) + "\n")

        if p1.returncode != 0 or p2.returncode != 0:
            raise RuntimeError(f"bcftools pipeline failed. See log: {log_path}")

def run_cmd(cmd, log_path=None):
    print("  $", " ".join(cmd))
    res = subprocess.run(cmd, capture_output=True, text=True)
    if log_path:
        with open(log_path, "a") as lf:
            lf.write("\n" + " ".join(cmd) + "\n")
            lf.write("STDOUT:\n" + (res.stdout or "") + "\n")
            lf.write("STDERR:\n" + (res.stderr or "") + "\n")
    else:
        if res.stdout.strip(): print(res.stdout.strip())
        if res.stderr.strip(): print(res.stderr.strip())
    if res.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(cmd)}")

# Tool checks
for tool in ("bcftools", "samtools"):
    if not have_tool(tool):
        raise RuntimeError(f"{tool} not found in PATH. Activate the conda env used for this notebook.")

# Find BAMs
bams = sorted(Path(ALN_DIR).glob("*.dedup.bam"))
if not bams:
    raise RuntimeError(f"No deduplicated BAMs found in {ALN_DIR}. Run the alignment cell first.")

# Ensure BAM indices exist
for b in bams:
    bai = Path(str(b) + ".bai")
    if not bai.exists():
        print(f"Indexing BAM: {b.name}")
        run_cmd(["samtools", "index", str(b)])

# Output paths
out_vcf_gz = VAR_DIR / "cohort.bcftools.vcf.gz"
out_vcf_tbi = VAR_DIR / "cohort.bcftools.vcf.gz.tbi"
log_path = LOGS_DIR / "bcftools_call.log"

# Idempotency check
if out_vcf_gz.exists() and out_vcf_tbi.exists():
    print(f"bcftools VCF already exists: {out_vcf_gz}")
else:
    print(f"Running bcftools (threads: {THREADS}) on {len(bams)} BAM(s)")
    # bcftools uses OpenMP/pthreads internally; we can hint via OMP/MKL vars (optional)
    os.environ["OMP_NUM_THREADS"] = str(THREADS)
    os.environ["OPENBLAS_NUM_THREADS"] = str(THREADS)
    os.environ["MKL_NUM_THREADS"] = str(THREADS)

    run_pipe_bcftools_mpileup_call(bams, out_vcf_gz, log_path)

    # Index the VCF
    print("Indexing VCF...")
    run_cmd(["bcftools", "index", "-t", str(out_vcf_gz)], log_path=log_path)

# Optional: basic stats
stats_txt = VAR_DIR / "cohort.bcftools.stats.txt"
if not stats_txt.exists():
    print("Computing bcftools stats...")
    run_cmd(["bcftools", "stats", "-F", str(REF_FASTA), "-s", "-", str(out_vcf_gz)])
    # bcftools stats prints to stdout; we can save separately if desired

print("\nbcftools calling complete.")
print("Output:", out_vcf_gz)

Running bcftools (threads: 16) on 2 BAM(s)
  $ bcftools mpileup -Ou -f /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/reference/GCA_036925085.1_qdBraProd1.0.pri_genomic.fa -q 20 -Q 20 -a FORMAT/DP,FORMAT/AD /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/alignments/CCGPMC021_BMEA101904_S1_L004001.dedup.bam /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/alignments/CCGPMC021_BMEA101907_S2_L004001.dedup.bam | bcftools call -mv -Oz -o /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/variants/bcftools/cohort.bcftools.vcf.gz
Indexing VCF...
  $ bcftools index -t /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/variants/bcftools/cohort.bcftools.vcf.gz
Computing bcftools stats...
  $ 

In [ ]:
## GATK gVCF per sample → Joint genotyping (output in a different directory from bcf tools above)
## This was killed on the first attempt because it took way too long to run - 
## so I need to think about using GATK - Is it just to slow - W
import os, shutil, subprocess
from pathlib import Path

# ----------- Inputs -----------
try:
    PROJECT_ROOT
except NameError:
    raise RuntimeError("PROJECT_ROOT not set. Run earlier setup cells.")

READ_SET_NAME = globals().get("READ_SET_NAME", None) or read_dir_dropdown.value
RUN_DIR   = Path(PROJECT_ROOT) / "results" / READ_SET_NAME
ALN_DIR   = RUN_DIR / "alignments"
GVCF_DIR  = RUN_DIR / "gvcf"
VAR_DIR   = RUN_DIR / "variants" / "gatk"
LOGS_DIR  = RUN_DIR / "logs"
for d in (GVCF_DIR, VAR_DIR, LOGS_DIR):
    d.mkdir(parents=True, exist_ok=True)

# From indexing cell
try:
    REF_FASTA
except NameError:
    raise RuntimeError("REF_FASTA is not defined. Run the indexing cell first.")

CPU_THREADS_SUGGESTED = globals().get("CPU_THREADS_SUGGESTED", None) or max(1, (os.cpu_count() or 2)//2)
# GATK HaplotypeCaller uses --native-pair-hmm-threads (often best <= 4)
HC_THREADS = max(1, min(int(CPU_THREADS_SUGGESTED), 4))

def have_tool(name: str) -> bool:
    return shutil.which(name) is not None

def run_cmd(cmd, log_path=None):
    print("  $", " ".join(cmd))
    res = subprocess.run(cmd, capture_output=True, text=True)
    if log_path:
        with open(log_path, "a") as lf:
            lf.write("\n" + " ".join(cmd) + "\n")
            lf.write("STDOUT:\n" + (res.stdout or "") + "\n")
            lf.write("STDERR:\n" + (res.stderr or "") + "\n")
    else:
        if res.stdout.strip(): print(res.stdout.strip())
        if res.stderr.strip(): print(res.stderr.strip())
    if res.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(cmd)}")

# Tool checks
if not have_tool("gatk"):
    raise RuntimeError("gatk not found. Activate the conda env with GATK.")

# Collect dedup BAMs
bam_paths = sorted(ALN_DIR.glob("*.dedup.bam"))
if not bam_paths:
    raise RuntimeError(f"No deduplicated BAMs found in {ALN_DIR}. Run the alignment cell first.")

# Ensure BAM indices exist
for b in bam_paths:
    bai = Path(str(b) + ".bai")
    if not bai.exists():
        print(f"Indexing BAM: {b.name}")
        run_cmd(["samtools", "index", str(b)])

# --- 1) Per-sample HaplotypeCaller to gVCF ---
sample_gvcfs = []
for bam in bam_paths:
    sample_id = bam.stem.replace(".dedup", "")
    gvcf = GVCF_DIR / f"{sample_id}.g.vcf.gz"
    gvcf_tbi = GVCF_DIR / f"{sample_id}.g.vcf.gz.tbi"
    log = LOGS_DIR / f"{sample_id}.haplotypecaller.log"

    if gvcf.exists() and gvcf_tbi.exists():
        print(f"[{sample_id}] gVCF exists, skipping HaplotypeCaller.")
    else:
        print(f"[{sample_id}] Running GATK HaplotypeCaller → gVCF (threads: {HC_THREADS})")
        cmd = [
            "gatk", "HaplotypeCaller",
            "-R", str(REF_FASTA),
            "-I", str(bam),
            "-O", str(gvcf),
            "-ERC", "GVCF",
            "--native-pair-hmm-threads", str(HC_THREADS),
        ]
        run_cmd(cmd, log_path=log)

        # Ensure index
        if not gvcf_tbi.exists():
            if shutil.which("bcftools"):
                run_cmd(["bcftools", "index", "-t", str(gvcf)], log_path=log)
            elif shutil.which("tabix"):
                run_cmd(["tabix", "-p", "vcf", str(gvcf)], log_path=log)
            else:
                print(f"WARNING: No bcftools/tabix found to index {gvcf}. Some downstream steps may fail.")

    sample_gvcfs.append(gvcf)

# --- 2) Combine GVCFs (skip if only one sample) ---
combined_gvcf = VAR_DIR / "combined.g.vcf.gz"
combined_tbi  = VAR_DIR / "combined.g.vcf.gz.tbi"

if len(sample_gvcfs) == 1:
    print("\nSingle-sample run: skipping CombineGVCFs and using the single gVCF directly for genotyping.")
    combined_gvcf = sample_gvcfs[0]
    combined_tbi  = Path(str(combined_gvcf) + ".tbi")
else:
    if combined_gvcf.exists() and combined_tbi.exists():
        print("\nCombined gVCF already exists, skipping CombineGVCFs.")
    else:
        print("\nCombining gVCFs with GATK CombineGVCFs...")
        cmd = ["gatk", "CombineGVCFs", "-R", str(REF_FASTA), "-O", str(combined_gvcf)]
        # Add each per-sample gVCF
        for gv in sample_gvcfs:
            cmd.extend(["--variant", str(gv)])
        run_cmd(cmd, log_path=LOGS_DIR / "combine_gvcfs.log")

        # Index combined
        if shutil.which("bcftools"):
            run_cmd(["bcftools", "index", "-t", str(combined_gvcf)])
        elif shutil.which("tabix"):
            run_cmd(["tabix", "-p", "vcf", str(combined_gvcf)])

# --- 3) Joint genotyping ---
cohort_vcf = VAR_DIR / "cohort.gatk.vcf.gz"
cohort_tbi = VAR_DIR / "cohort.gatk.vcf.gz.tbi"

if cohort_vcf.exists() and cohort_tbi.exists():
    print("\nGenotyped VCF already exists, skipping GenotypeGVCFs.")
else:
    print("\nRunning GATK GenotypeGVCFs...")
    run_cmd(["gatk", "GenotypeGVCFs",
             "-R", str(REF_FASTA),
             "-V", str(combined_gvcf),
             "-O", str(cohort_vcf)],
            log_path=LOGS_DIR / "genotype_gvcfs.log")

    # Index the final VCF
    if shutil.which("bcftools"):
        run_cmd(["bcftools", "index", "-t", str(cohort_vcf)])
    elif shutil.which("tabix"):
        run_cmd(["tabix", "-p", "vcf", str(cohort_vcf)])

print("\nGATK pipeline complete.")
print("Per-sample gVCFs:", GVCF_DIR)
print("Cohort VCF:", cohort_vcf)

In [ ]:
# 6. Variant Calling
# Two common options:
# Option A: bcftools
#bcftools mpileup -Ou -f reference.fasta sample.dedup.bam | \
#bcftools call -mv -Oz -o sample.vcf.gz
#bcftools index sample.vcf.gz

In [ ]:
# Option B: GATK HaplotypeCaller
#gatk HaplotypeCaller -R reference.fasta -I sample.dedup.bam -O sample.g.vcf.gz -ERC GVCF

In [ ]:
# 7. (Optional) Joint Genotyping for Multiple Samples
#gatk CombineGVCFs -R reference.fasta --variant sample1.g.vcf.gz --variant sample2.g.vcf.gz -O cohort.g.vcf.gz
#gatk GenotypeGVCFs -R reference.fasta -V cohort.g.vcf.gz -O cohort.vcf.gz

In [16]:
### Final Summary Cell
import os, re, json, shutil, subprocess
from pathlib import Path
from datetime import datetime

# ---------- Inputs from earlier cells ----------
try:
    PROJECT_ROOT
except NameError:
    raise RuntimeError("PROJECT_ROOT not set. Run the selection/setup cells first.")

READ_SET_NAME = globals().get("READ_SET_NAME") or read_dir_dropdown.value
PAIRS         = globals().get("PAIRS", {})
REF_FASTA     = globals().get("REF_FASTA", None)

RUN_DIR   = Path(PROJECT_ROOT) / "results" / READ_SET_NAME
QC_RAW    = RUN_DIR / "qc" / "fastqc_raw"
QC_TRIM   = RUN_DIR / "qc" / "fastqc_trimmed"
TRIM_DIR  = RUN_DIR / "trimmed"
ALN_DIR   = RUN_DIR / "alignments"
VAR_DIR   = RUN_DIR / "variants"
LOGS_DIR  = RUN_DIR / "logs"
REPORT_MD = RUN_DIR / "run_report.md"

# Expected VCF (adjust name if you used a different filename)
VCF_GZ   = VAR_DIR / "bcftools" / "cohort.bcftools.vcf.gz"
VCF_TBI  = VAR_DIR /  "bcftools" / "cohort.bcftools.vcf.gz.tbi"

# ---------- Tool checks ----------
def have_tool(name: str) -> bool:
    return shutil.which(name) is not None

if not have_tool("samtools"):
    raise RuntimeError("samtools not found in PATH.")
if not have_tool("bcftools"):
    raise RuntimeError("bcftools not found in PATH.")

def run_cmd(cmd: list[str]) -> str:
    """Run command and return stdout; raise if non-zero exit."""
    proc = subprocess.run(cmd, capture_output=True, text=True)
    if proc.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(cmd)}\n{proc.stderr}")
    return proc.stdout

# ---------- Alignment stats per sample ----------
align_stats = {}
for sample_id in sorted(PAIRS.keys()):
    bam_dedup = ALN_DIR / f"{sample_id}.dedup.bam"
    md_metrics = ALN_DIR / f"{sample_id}.markdup.metrics.txt"
    if not bam_dedup.exists():
        print(f"[WARN] Missing dedup BAM for sample {sample_id}: {bam_dedup}")
        continue
    # samtools flagstat summary
    fs = run_cmd(["samtools", "flagstat", str(bam_dedup)])
    # Parse a few key values
    # Typical lines:
    #   123456 + 0 in total (QC-passed reads + QC-failed reads)
    #   120000 + 0 primary
    #   100000 + 0 mapped (81.30% : N/A)
    #   90000 + 0 properly paired (75.00% : N/A)
    #   5000 + 0 duplicates
    total = mapped = prop = dups = None
    for line in fs.splitlines():
        if " in total " in line:
            m = re.match(r"(\d+)\s+\+\s+\d+\s+in total", line)
            if m: total = int(m.group(1))
        elif " mapped (" in line:
            m1 = re.match(r"(\d+)\s+\+\s+\d+\s+mapped\s+\(([\d\.]+)%", line)
            if m1:
                mapped = (int(m1.group(1)), float(m1.group(2)))
        elif " properly paired " in line:
            m2 = re.match(r"(\d+)\s+\+\s+\d+\s+properly paired\s+\(([\d\.]+)%", line)
            if m2:
                prop = (int(m2.group(1)), float(m2.group(2)))
        elif " duplicates" in line:
            m3 = re.match(r"(\d+)\s+\+\s+\d+\s+duplicates", line)
            if m3: dups = int(m3.group(1))

    # Parse duplication rate from MarkDuplicates metrics (Picard/GATK)
    dup_rate = None
    if md_metrics.exists():
        # Picard metrics contain a header section then a table with PERCENT_DUPLICATION
        txt = Path(md_metrics).read_text(errors="ignore").splitlines()
        header_idx = None
        for i, ln in enumerate(txt):
            if ln.strip().startswith("LIBRARY\t"):
                header_idx = i
                break
        if header_idx is not None and header_idx + 1 < len(txt):
            header = txt[header_idx].split("\t")
            data = txt[header_idx+1].split("\t")
            if "PERCENT_DUPLICATION" in header and len(data) == len(header):
                j = header.index("PERCENT_DUPLICATION")
                try:
                    dup_rate = float(data[j])
                except Exception:
                    pass

    align_stats[sample_id] = {
        "total_reads": total,
        "mapped_reads": mapped[0] if mapped else None,
        "mapped_pct": mapped[1] if mapped else None,
        "proper_pairs": prop[0] if prop else None,
        "proper_pairs_pct": prop[1] if prop else None,
        "dup_reads": dups,
        "dup_rate_metric": dup_rate,  # from MarkDuplicates (0..1)
        "bam": str(bam_dedup),
    }

# ---------- VCF-level stats ----------
vcf_stats = {
    "vcf_path": str(VCF_GZ),
    "indexed": VCF_TBI.exists(),
    "samples": [],
    "n_variants": None,
    "n_snps": None,
    "n_indels": None,
    "ts_tv": None
}

if VCF_GZ.exists():
    # Samples
    vcf_stats["samples"] = run_cmd(["bcftools", "query", "-l", str(VCF_GZ)]).split()
    # bcftools stats
    stats_txt = run_cmd(["bcftools", "stats", str(VCF_GZ)])
    # Parse:
    # SN    0    number of records:    12345
    # SN    0    number of SNPs:    10000
    # SN    0    number of indels:    2345
    # TSTV    ...
    for line in stats_txt.splitlines():
        if line.startswith("SN"):
            parts = line.split("\t")
            if len(parts) >= 4:
                label = parts[2].strip().lower()
                val = parts[3].strip()
                if label == "number of records:":
                    vcf_stats["n_variants"] = int(val)
                elif label == "number of snps:":
                    vcf_stats["n_snps"] = int(val)
                elif label == "number of indels:":
                    vcf_stats["n_indels"] = int(val)
        elif line.startswith("TSTV"):
            # TSTV    ALL    nTransitions    nTransversions    ts/tv
            parts = line.split("\t")
            if len(parts) >= 5 and parts[1] == "all".upper() or parts[1] == "ALL":
                try:
                    vcf_stats["ts_tv"] = float(parts[4])
                except Exception:
                    pass
else:
    print(f"[WARN] VCF not found: {VCF_GZ}")

# ---------- Disk usage info ----------
def human_bytes(n):
    units = ["B","KB","MB","GB","TB"]
    i = 0
    f = float(n)
    while f >= 1024 and i < len(units)-1:
        f /= 1024; i += 1
    return f"{f:.2f} {units[i]}"

def total_size(path: Path) -> int:
    total = 0
    if not path.exists(): return 0
    for root, _, files in os.walk(path):
        for f in files:
            p = Path(root) / f
            try:
                total += p.stat().st_size
            except Exception:
                pass
    return total

disk = shutil.disk_usage(str(RUN_DIR))
sizes = {
    "trimmed": total_size(TRIM_DIR),
    "alignments": total_size(ALN_DIR),
    "variants": total_size(VAR_DIR),
    "qc_raw": total_size(QC_RAW),
    "qc_trim": total_size(QC_TRIM),
}

# ---------- Build Markdown report ----------
lines = []
lines.append(f"# Run Report — {READ_SET_NAME}\n")
lines.append(f"_Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}_\n")
lines.append("## Paths")
lines.append(f"- Project root: `{PROJECT_ROOT}`")
lines.append(f"- Run dir: `{RUN_DIR}`")
if REF_FASTA: lines.append(f"- Reference (run-local): `{REF_FASTA}`")
lines.append("")

lines.append("## Input summary")
lines.append(f"- Samples detected: **{len(PAIRS)}**")
lines.append(f"- Trimmed reads dir: `{TRIM_DIR}`")
lines.append("")

lines.append("## Alignment summary (per sample)")
if align_stats:
    lines.append("| Sample | Total reads | Mapped (%) | Proper pairs (%) | Duplicates | Dup. rate (MarkDup) | BAM |")
    lines.append("|---|---:|---:|---:|---:|---:|---|")
    for sid, st in align_stats.items():
        m_pct = f"{st['mapped_pct']:.2f}%" if st['mapped_pct'] is not None else "NA"
        p_pct = f"{st['proper_pairs_pct']:.2f}%" if st['proper_pairs_pct'] is not None else "NA"
        dup_reads = f"{st['dup_reads']:,}" if st['dup_reads'] is not None else "NA"
        dup_rate = f"{st['dup_rate_metric']*100:.2f}%" if st['dup_rate_metric'] is not None else "NA"
        lines.append(f"| {sid} | {st['total_reads'] or 'NA'} | {m_pct} | {p_pct} | {dup_reads} | {dup_rate} | `{st['bam']}` |")
else:
    lines.append("_No alignment stats found. Did the alignment cell run?_")
lines.append("")

lines.append("## VCF summary")
if VCF_GZ.exists():
    lines.append(f"- VCF: `{VCF_GZ.name}` (indexed: **{ 'yes' if vcf_stats['indexed'] else 'no' }**)")
    lines.append(f"- Samples in VCF: **{len(vcf_stats['samples'])}**")
    lines.append(f"- Total variants: **{vcf_stats['n_variants'] if vcf_stats['n_variants'] is not None else 'NA'}**")
    lines.append(f"- SNPs: **{vcf_stats['n_snps'] if vcf_stats['n_snps'] is not None else 'NA'}**")
    lines.append(f"- Indels: **{vcf_stats['n_indels'] if vcf_stats['n_indels'] is not None else 'NA'}**")
    lines.append(f"- Ts/Tv: **{vcf_stats['ts_tv'] if vcf_stats['ts_tv'] is not None else 'NA'}**")
else:
    lines.append(f"- VCF not found at `{VCF_GZ}`")
lines.append("")

lines.append("## Disk usage for this run")
lines.append(f"- Run dir free/total: {human_bytes(disk.free)} / {human_bytes(disk.total)}")
for k, v in sizes.items():
    lines.append(f"- {k}: {human_bytes(v)}")
lines.append("")

# Write report
REPORT_MD.write_text("\n".join(lines))
print(REPORT_MD.read_text())

# Also produce a machine-readable manifest (optional)
manifest = {
    "read_set": READ_SET_NAME,
    "run_dir": str(RUN_DIR),
    "reference": str(REF_FASTA) if REF_FASTA else None,
    "samples": sorted(PAIRS.keys()),
    "alignment": align_stats,
    "vcf": vcf_stats,
    "sizes": {k: int(v) for k, v in sizes.items()},
    "generated_at": datetime.now().isoformat()
}
with open(RUN_DIR / "run_manifest.json", "w") as jf:
    json.dump(manifest, jf, indent=2)
print(f"\nSaved:\n- {REPORT_MD}\n- {RUN_DIR / 'run_manifest.json'}")


# Run Report — Brachycybe_producta_test_readset

_Generated: 2026-01-21 16:45:39_

## Paths
- Project root: `/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline`
- Run dir: `/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset`
- Reference (run-local): `/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/reference/GCA_036925085.1_qdBraProd1.0.pri_genomic.fa`

## Input summary
- Samples detected: **2**
- Trimmed reads dir: `/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/trimmed`

## Alignment summary (per sample)
| Sample | Total reads | Mapped (%) | Proper pairs (%) | Duplicates | Dup. rate (MarkDup) | BAM |
|---|---:|---:|---:|---:|---:|---|
| CCGPMC021_BMEA101904_S1_L004001 | 22722817 | 93.99% | 85.39% | 2,764,728 | 13.35% | `/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pi

In [19]:
# 📈 Quick VCF QC plots: Ti/Tv overall + by contig, DP and QUAL histograms
import os, subprocess, shutil
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Choose which VCF to visualize
VAR_DIR = Path(PROJECT_ROOT) / "results" / READ_SET_NAME / "variants"
vcf_candidates = [
#    VAR_DIR / "cohort.gatk.genotyped.vcf.gz",
    VAR_DIR / "bcftools" / "cohort.bcftools.vcf.gz"
]
vcf = next((v for v in vcf_candidates if v.exists()), None)
if vcf is None:
    raise RuntimeError("No cohort VCF found. Run bcftools or GATK variant calling first.")

if shutil.which("bcftools") is None:
    raise RuntimeError("bcftools not in PATH; required for bcftools stats.")

stats_path = vcf.with_suffix(".stats.txt")
if not stats_path.exists():
    print("Running bcftools stats...")
    subprocess.run(["bcftools", "stats", "-F", str(REF_FASTA), "-s", "-", str(vcf), "-o", str(stats_path)], check=True)

# Parse bcftools stats sections
# SN lines: summary numbers including TSTV
# TSTV lines: per-chrom Ts/Tv
sn_rows = []
tstv_rows = []
with open(stats_path) as f:
    for line in f:
        if line.startswith("SN"):
            # SN  <col>  key  value
            parts = line.strip().split("\t")
            if len(parts) >= 4:
                sn_rows.append((parts[2], float(parts[3]) if parts[3].replace('.','',1).isdigit() else parts[3]))
        elif line.startswith("TSTV"):
            # TSTV  <col>  chrom  ts tv ratio
            parts = line.strip().split("\t")
            if len(parts) >= 6:
                chrom = parts[2]
                ts = float(parts[3]); tv = float(parts[4]); ratio = float(parts[5]) if parts[5] not in ("nan", ".") else float("nan")
                tstv_rows.append((chrom, ts, tv, ratio))

sn_df = pd.DataFrame(sn_rows, columns=["metric", "value"])
tstv_df = pd.DataFrame(tstv_rows, columns=["chrom", "ts", "tv", "tstv"])

# Pull some handy SN metrics
overall_tstv = float(sn_df.loc[sn_df["metric"] == "TSTV", "value"].iloc[0]) if (sn_df["metric"] == "TSTV").any() else None
n_snp = int(sn_df.loc[sn_df["metric"] == "number of SNPs", "value"].iloc[0]) if (sn_df["metric"] == "number of SNPs").any() else None

# Depth & QUAL quick histograms via bcftools query → pandas (avoid heavy VCF parsing libs)
# DP field: FORMAT/DP (per-sample) and/or INFO/DP (site); we’ll try INFO/DP first
import tempfile, numpy as np
tmp_dp = Path(tempfile.mkstemp(prefix="dp_", suffix=".txt")[1])
tmp_qual = Path(tempfile.mkstemp(prefix="qual_", suffix=".txt")[1])

# Extract INFO/DP (may be missing; then array will be empty)
subprocess.run(f'bcftools query -f "%INFO/DP\\n" {vcf} | grep -E "^[0-9]+$" > {tmp_dp}', shell=True, check=False)
# Extract QUAL (column 6)
subprocess.run(f'bcftools query -f "%QUAL\\n" {vcf} | grep -E "^[0-9]+(\\.[0-9]+)?$" > {tmp_qual}', shell=True, check=False)

dp_vals = np.loadtxt(tmp_dp, dtype=float) if tmp_dp.stat().st_size > 0 else np.array([])
qual_vals = np.loadtxt(tmp_qual, dtype=float) if tmp_qual.stat().st_size > 0 else np.array([])
tmp_dp.unlink(missing_ok=True); tmp_qual.unlink(missing_ok=True)

# ---- Plot ----
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
if overall_tstv is not None:
    plt.title(f"Overall Ti/Tv = {overall_tstv:.2f}\nSNPs={n_snp if n_snp is not None else 'NA'}")
else:
    plt.title("Overall Ti/Tv (NA)")
plt.axis('off')

plt.subplot(1, 3, 2)
if not tstv_df.empty:
    tstv_plot = tstv_df.sort_values("tstv", ascending=False).head(20)
    plt.barh(tstv_plot["chrom"], tstv_plot["tstv"], color="#4e79a7")
    plt.xlabel("Ti/Tv by contig (top 20)")
    plt.gca().invert_yaxis()
else:
    plt.text(0.5, 0.5, "No per-contig Ti/Tv", ha="center")

plt.subplot(1, 3, 3)
if dp_vals.size > 0:
    vmax = np.percentile(dp_vals, 99)
    plt.hist(dp_vals[dp_vals <= vmax], bins=50, color="#59a14f")
    plt.xlabel("INFO/DP (site depth)")
    plt.ylabel("Count")
    plt.title("Depth distribution (99th pct clipped)")
else:
    plt.text(0.5, 0.5, "No DP values", ha="center")

plt.tight_layout()
plt.show()

# QUAL histogram (separate)
plt.figure(figsize=(5,4))
if qual_vals.size > 0:
    qmax = np.percentile(qual_vals, 99)
    plt.hist(qual_vals[qual_vals <= qmax], bins=50, color="#e15759")
    plt.xlabel("QUAL")
    plt.ylabel("Count")
    plt.title("QUAL distribution (99th pct clipped)")
else:
    plt.text(0.5, 0.5, "No QUAL values", ha="center")
plt.tight_layout()
plt.show()

print("Tip: On chromosome-scale assemblies, evaluate Ti/Tv per chromosome and compare between autosomes vs. known/putative sex chromosomes if applicable. Low Ti/Tv in specific regions may indicate repeats or low mappability.")


Running bcftools stats...


stats: invalid option -- 'o'

About:   Parses VCF or BCF and produces stats which can be plotted using plot-vcfstats.
         When two files are given, the program generates separate stats for intersection
         and the complements. By default only sites are compared, -s/-S must given to include
         also sample columns.
Usage:   bcftools stats [options] <A.vcf.gz> [<B.vcf.gz>]

Options:
        --af-bins LIST               Allele frequency bins, a list (0.1,0.5,1) or a file (0.1\n0.5\n1)
        --af-tag STRING              Allele frequency tag to use, by default estimated from AN,AC or GT
    -1, --1st-allele-only            Include only 1st allele at multiallelic sites
    -c, --collapse STRING            Treat as identical records with <snps|indels|both|all|some|none>, see man page for details [none]
    -d, --depth INT,INT,INT          Depth distribution: min,max,bin size [0,500,1]
    -e, --exclude EXPR               Exclude sites for which the expression is true (see man

CalledProcessError: Command '['bcftools', 'stats', '-F', '/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/reference/GCA_036925085.1_qdBraProd1.0.pri_genomic.fa', '-s', '-', '/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/variants/bcftools/cohort.bcftools.vcf.gz', '-o', '/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/results/Brachycybe_producta_test_readset/variants/bcftools/cohort.bcftools.vcf.stats.txt']' returned non-zero exit status 1.

## Utility cells - put here for storage

In [ ]:

# tests/make_tiny_dataset.py
#
# Generates a tiny reference + two paired-end samples.
# - Sample1 matches reference
# - Sample2 has a few SNPs on ctgA
# - A fraction of reads include real adapter tails (i7 on R1 3', i5 on R2 3')
# - Outputs gzipped FASTQs
#
# Uses real adapters from:
#   /group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline/data/adapters/adapters.fasta
#
# After running, you can symlink/copy:
#   tests/tiny/ref/tiny.fa        -> data/references/tiny.fa
#   tests/tiny/reads/*.fastq.gz   -> data/read_directories/tiny_test/
#
from pathlib import Path
import random
import gzip
import os
import re

# -----------------------------
# Configuration
# -----------------------------
PROJECT_ROOT = Path("/group/jbondgrp2/stephenRichards/_Analysis_projects/_Ems_vcf_pipeline")

ADAPTERS_REAL = PROJECT_ROOT / "data" / "adapters" / "adapters.fasta"
OUT_ROOT      = Path("tests") / "tiny"
REF_DIR       = OUT_ROOT / "ref"
READS_DIR     = OUT_ROOT / "reads"
ADAPT_DIR     = OUT_ROOT / "adapters"    # optional local reference to adapters for the tiny test

REF_NAME      = "tiny.fa"
SAMPLES       = ["sample1", "sample2"]

READ_LEN      = 100     # length per end
INSERT_MEAN   = 300     # typical insert length
N_PAIRS       = 3000    # pairs per sample; keep small to run fast
ADAPT_FRAC    = 0.30    # fraction of pairs that will include adapter tails

RNG_SEED      = 42

# -----------------------------
# Helpers
# -----------------------------
def rand_dna(n):
    return "".join(random.choice("ACGT") for _ in range(n))

def rc(seq):
    comp = str.maketrans("ACGT", "TGCA")
    return seq.translate(comp)[::-1]

def load_adapters(fa_path):
    """
    Parse a simple FASTA file containing:
      >i5
      <seq>
      >i7
      <seq>
    Returns dict {'i5': seq, 'i7': seq}
    """
    if not fa_path.exists():
        raise FileNotFoundError(f"Adapters FASTA not found at: {fa_path}")

    adapters = {}
    name = None
    seqs = []
    with open(fa_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if name and seqs:
                    adapters[name] = "".join(seqs).upper()
                name = line[1:].strip().split()[0]  # take token after '>'
                seqs = []
            else:
                seqs.append(line.strip())
        if name and seqs:
            adapters[name] = "".join(seqs).upper()

    # Normalize keys: we expect 'i5' and 'i7'
    norm = {}
    for k, v in adapters.items():
        key = k.lower()
        if key in ("i5", "i7"):
            norm[key] = re.sub(r"[^ACGT]", "", v.upper())
    if "i5" not in norm or "i7" not in norm:
        raise ValueError(f"Adapters file must contain >i5 and >i7 entries. Found keys: {list(norm.keys())}")
    return norm

def write_fastq_gz(path, records):
    with gzip.open(path, "wt") as f:
        for name, seq, qual in records:
            f.write(f"@{name}\n{seq}\n+\n{qual}\n")

# -----------------------------
# Main
# -----------------------------
def main():
    random.seed(RNG_SEED)
    REF_DIR.mkdir(parents=True, exist_ok=True)
    READS_DIR.mkdir(parents=True, exist_ok=True)
    ADAPT_DIR.mkdir(parents=True, exist_ok=True)

    # Load real adapters
    adapters = load_adapters(ADAPTERS_REAL)
    i5 = adapters["i5"]
    i7 = adapters["i7"]

    # Symlink/copy adapters into tiny test dir (optional convenience)
    target_adapt = ADAPT_DIR / "adapters.fasta"
    if not target_adapt.exists():
        try:
            target_adapt.symlink_to(ADAPTERS_REAL)
        except Exception:
            # Fallback: copy
            target_adapt.write_text(ADAPTERS_REAL.read_text())

    # Build a tiny reference
    contigs = {
        "ctgA": list(rand_dna(30000)),
        "ctgB": list(rand_dna(20000)),
    }

    # SNP sites that sample2 will carry as ALT (ref stays as-is)
    snp_sites = [1000, 5000, 12000, 18000, 25000]  # on ctgA
    # For sample2 we’ll flip bases at these positions (simulate true polymorphisms)
    alt_bases = {}
    for pos in snp_sites:
        refb = contigs["ctgA"][pos]
        alts = [b for b in "ACGT" if b != refb]
        altb = random.choice(alts)
        alt_bases[pos] = altb

    ref_path = REF_DIR / REF_NAME
    with open(ref_path, "w") as f:
        for name, seq_list in contigs.items():
            f.write(f">{name}\n")
            seq = "".join(seq_list)
            for i in range(0, len(seq), 60):
                f.write(seq[i:i+60] + "\n")

    print(f"[ok] Wrote reference: {ref_path}")

    # Build per-sample sequence dicts (strings)
    seqs_s1 = {k: "".join(v) for k, v in contigs.items()}
    # Sample2: apply ALT at snp_sites on ctgA (only to sample2 basis)
    s2_ctgA = list(seqs_s1["ctgA"])
    for pos, alt in alt_bases.items():
        s2_ctgA[pos] = alt
    seqs_s2 = dict(seqs_s1)
    seqs_s2["ctgA"] = "".join(s2_ctgA)

    # Generate paired reads with a fraction containing adapter tails.
    # Model: For ADAPT_FRAC of fragments, create an insert shorter than READ_LEN,
    # so R1 and R2 have 3' overhangs filled by adapters (R1 gets i7 tail; R2 gets i5 tail).
    def generate_pairs(sample, seqs, n_pairs=N_PAIRS, read_len=READ_LEN, insert_mean=INSERT_MEAN, adapt_frac=ADAPT_FRAC):
        qual = "I" * read_len  # simple high-quality score
        r1_records = []
        r2_records = []

        keys = list(seqs.keys())
        for idx in range(n_pairs):
            ctg = random.choice(keys)
            seq = seqs[ctg]

            # Choose insert length
            if random.random() < adapt_frac:
                # short fragment (force adapter tails)
                frag_len = random.randint(30, read_len - 10)  # 20..90-ish
            else:
                # near mean with some variance
                frag_len = max(read_len + 20, int(random.gauss(insert_mean, insert_mean * 0.1)))

            if len(seq) <= frag_len + 10:
                # pick another contig if tiny; if still small, clamp
                frag_len = min(frag_len, len(seq) - 10)

            start = random.randint(0, max(0, len(seq) - frag_len - 1))
            frag = seq[start:start+frag_len]

            # R1 = forward 5'->3' from frag start
            r1seq = frag[:read_len]
            # R2 = reverse complement from frag end
            r2seq = rc(frag[-read_len:])

            # If fragment is shorter than read length, append adapter tails on 3'
            if len(frag) < read_len:
                overhang = read_len - len(frag)
                # R1 3' tail with i7
                r1_tail = (i7 * ((overhang // len(i7)) + 1))[:overhang]
                r1seq = frag + r1_tail

                # R2 3' tail with i5 (remember r2seq is RC of end of frag; appending tail in sequence-space)
                r2_tail = (i5 * ((overhang // len(i5)) + 1))[:overhang]
                # Because r2seq is already reverse-complement of fragment end, to simulate a real adapter tail
                # on the 3' end in read-space, we append the adapter *as-is* (not RC) to r2seq.
                r2seq = r2seq + r2_tail

            name = f"{sample}_{ctg}_{start}"
            r1_records.append((name + "/1", r1seq, qual))
            r2_records.append((name + "/2", r2seq, qual))

        return r1_records, r2_records

    r1_s1, r2_s1 = generate_pairs("sample1", seqs_s1)
    r1_s2, r2_s2 = generate_pairs("sample2", seqs_s2)

    # Write gzipped FASTQs
    out_r1_s1 = READS_DIR / "sample1_R1.fastq.gz"
    out_r2_s1 = READS_DIR / "sample1_R2.fastq.gz"
    out_r1_s2 = READS_DIR / "sample2_R1.fastq.gz"
